In [1]:
import os
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
def read(file_name):
    data_dir = '/Users/keiji/work/kaggle/sales1c/'
    pickle = data_dir + file_name + '.pickle'
    if os.path.exists(pickle):
        return pd.read_pickle(pickle)
    df = pd.read_csv(data_dir + file_name)
    df.to_pickle(pickle)
    return df

df_icats = read('item_categories.csv')
df_items = read('items.csv')
df_shops = read('shops.csv')
df_test = read('test.csv.gz')
df_train_all = read('sales_train.csv.gz')

In [3]:
TARGET_DATE_BLOCK = 32
df_train = df_train_all[df_train_all.date_block_num < TARGET_DATE_BLOCK]
df_val = df_train_all[df_train_all.date_block_num == TARGET_DATE_BLOCK]

# Training data
All non-null.
3M rows.

In [4]:
df_train_all[:1000000].info()
print()
df_train_all[1000000:2000000].info()
print()
df_train_all[2000000:].info()

df_train_all.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 6 columns):
date              1000000 non-null object
date_block_num    1000000 non-null int64
shop_id           1000000 non-null int64
item_id           1000000 non-null int64
item_price        1000000 non-null float64
item_cnt_day      1000000 non-null float64
dtypes: float64(2), int64(3), object(1)
memory usage: 45.8+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 1000000 to 1999999
Data columns (total 6 columns):
date              1000000 non-null object
date_block_num    1000000 non-null int64
shop_id           1000000 non-null int64
item_id           1000000 non-null int64
item_price        1000000 non-null float64
item_cnt_day      1000000 non-null float64
dtypes: float64(2), int64(3), object(1)
memory usage: 45.8+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 935849 entries, 2000000 to 2935848
Data columns (total 6 columns):
date              935849

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


In [5]:
df_train_all.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


In [37]:
print('item_price mean+100std=', df_train_all.item_price.mean() + 100 * df_train_all.item_price.std())
df_train_all.sort_values(by='item_price', ascending=False).head(10)

item_price mean+100std= 173870.81630399663


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
1163158,13.12.2013,11,12,6066,307980.0,1.0
885138,17.09.2013,8,12,11365,59200.0,1.0
1488135,20.03.2014,14,25,13199,50999.0,1.0
2327159,29.01.2015,24,12,7241,49782.0,1.0
2931380,20.10.2015,33,22,13403,42990.0,1.0
2917760,20.10.2015,33,3,13403,42990.0,1.0
2910156,29.10.2015,33,12,13403,42990.0,1.0
2885692,23.10.2015,33,42,13403,42990.0,1.0
1006638,24.10.2013,9,12,7238,42000.0,1.0
2910155,20.10.2015,33,12,13403,41990.0,1.0


In [50]:
df_train_all[df_train_all.shop_id==12].describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,34694.000000,34694.0,34694.000000,34694.000000,34694.000000
mean,17.484407,12.0,9891.584770,1464.391142,2.117888
std,9.643071,0.0,6481.179913,3142.064428,16.500692
min,0.000000,12.0,27.000000,0.908714,-22.000000
25%,10.000000,12.0,3938.000000,300.000000,1.000000
50%,18.000000,12.0,9396.000000,720.000000,1.000000
75%,25.000000,12.0,15270.000000,1699.000000,1.000000
max,33.000000,12.0,22168.000000,307980.000000,2169.000000


In [49]:
df_train_all[df_train_all.item_id==20949].sort_values(by='item_price', ascending=False).head(10)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
352379,30.04.2013,3,22,20949,5.0,12.0
2134454,29.11.2014,22,56,20949,5.0,7.0
2119137,08.11.2014,22,28,20949,5.0,34.0
2119136,09.11.2014,22,28,20949,5.0,19.0
2119135,10.11.2014,22,28,20949,5.0,4.0
2119134,11.11.2014,22,28,20949,5.0,9.0
2119133,12.11.2014,22,28,20949,5.0,5.0
2119132,13.11.2014,22,28,20949,5.0,17.0
2119114,14.11.2014,22,28,20949,5.0,22.0
2119113,15.11.2014,22,28,20949,5.0,30.0


In [47]:
df_test[df_test.item_id == 6066]

,ID,shop_id,item_id
460,460,5,20949
5560,5560,4,20949
10660,10660,6,20949
15760,15760,3,20949
20860,20860,2,20949
25960,25960,7,20949
31060,31060,10,20949
36160,36160,12,20949
41260,41260,28,20949
46360,46360,31,20949


In [39]:
print('item_cnt_day mean+100std=', df_train_all.item_cnt_day.mean() + 100 * df_train_all.item_cnt_day.std())
df_train_all.sort_values(by='item_cnt_day', ascending=False).head(10)

item_cnt_day mean+100std= 263.1260839750612


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2909818,28.10.2015,33,12,11373,0.908714,2169.0
2326930,15.01.2015,24,12,20949,4.000000,1000.0
2864235,30.09.2015,32,12,9248,1692.526158,669.0
2851091,30.09.2015,32,55,9249,1702.825746,637.0
2608040,14.04.2015,27,12,3731,1904.548077,624.0
2626181,19.05.2015,28,12,11373,155.192950,539.0
2851073,29.09.2015,32,55,9249,1500.000000,533.0
2048518,02.10.2014,21,12,9242,1500.000000,512.0
2067669,09.10.2014,21,55,19437,899.000000,508.0
2864260,29.09.2015,32,12,9248,1500.000000,504.0


In [43]:
df_test[(df_test.shop_id == 12) & (df_test.item_id == 11373)]

,ID,shop_id,item_id
37296,37296,12,11373


In [44]:
df_train_all[(df_train_all.shop_id == 12) & (df_train_all.item_id == 11373)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
58323,08.01.2013,0,12,11373,1000.000000,1.0
58324,31.01.2013,0,12,11373,97.800000,5.0
176790,01.02.2013,1,12,11373,229.000000,1.0
176791,04.02.2013,1,12,11373,130.000000,1.0
302468,29.03.2013,2,12,11373,139.500000,2.0
302469,27.03.2013,2,12,11373,146.000000,1.0
302470,14.03.2013,2,12,11373,130.000000,1.0
302471,13.03.2013,2,12,11373,287.666667,3.0
302472,12.03.2013,2,12,11373,145.833333,12.0
302473,11.03.2013,2,12,11373,323.000000,2.0


# Test data
All non-null.
200k rows

In [6]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 3 columns):
ID         214200 non-null int64
shop_id    214200 non-null int64
item_id    214200 non-null int64
dtypes: int64(3)
memory usage: 4.9 MB


In [7]:
df_test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


# Items

In [51]:
df_items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [52]:
df_icats.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4
